In [1]:
import pandas as pd

# Carregando os datasets
stations = pd.read_csv('station.csv')
trips = pd.read_csv('trip.csv')

In [5]:
# Contagem de viagens de início e fim
start_station_usage = trips.groupby('start_station_name').size().reset_index(name='start_count')
end_station_usage = trips.groupby('end_station_name').size().reset_index(name='end_count')

# Juntar os contadores de uso por estação
station_usage = pd.merge(start_station_usage, end_station_usage, left_on='start_station_name', right_on='end_station_name', how='outer')

# Preencher valores NaN com 0 e somar os totais
station_usage.fillna(0, inplace=True)
station_usage['total_usage'] = station_usage['start_count'] + station_usage['end_count']


In [8]:
# Verifica se há valores nulos nas colunas de latitude e longitude
print(trips[['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude']].isnull().sum())


start_station_latitude     16668
start_station_longitude    16668
end_station_latitude       16668
end_station_longitude      16668
dtype: int64


In [9]:
# Remove as linhas que contêm valores nulos nas colunas de latitude e longitude
trips = trips.dropna(subset=['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude'])


In [10]:
# Substitui os valores nulos nas colunas de latitude e longitude por algum valor padrão (por exemplo, 0)
trips[['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude']] = trips[['start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude']].fillna(0)


In [12]:
from geopy.distance import geodesic

# Função para calcular a distância entre duas coordenadas
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

for index, trip in trips.iterrows():
    distance = calculate_distance(trip['start_station_latitude'], trip['start_station_longitude'],
                                  trip['end_station_latitude'], trip['end_station_longitude'])
    print(f"A distância da viagem {index} é {distance:.2f} quilômetros.")


A distância da viagem 0 é 2.24 quilômetros.
A distância da viagem 1 é 2.41 quilômetros.
A distância da viagem 2 é 2.41 quilômetros.
A distância da viagem 3 é 1.54 quilômetros.
A distância da viagem 4 é 1.37 quilômetros.
A distância da viagem 5 é 0.00 quilômetros.
A distância da viagem 6 é 0.69 quilômetros.
A distância da viagem 7 é 0.69 quilômetros.
A distância da viagem 8 é 0.69 quilômetros.
A distância da viagem 9 é 0.69 quilômetros.
A distância da viagem 10 é 0.69 quilômetros.
A distância da viagem 11 é 0.69 quilômetros.
A distância da viagem 12 é 0.69 quilômetros.
A distância da viagem 13 é 0.88 quilômetros.
A distância da viagem 14 é 0.86 quilômetros.
A distância da viagem 15 é 0.86 quilômetros.
A distância da viagem 16 é 0.86 quilômetros.
A distância da viagem 17 é 0.86 quilômetros.
A distância da viagem 18 é 0.86 quilômetros.
A distância da viagem 19 é 0.86 quilômetros.
A distância da viagem 20 é 0.86 quilômetros.
A distância da viagem 21 é 0.86 quilômetros.
A distância da viage

In [14]:
# Etapa 3: Calcular o Volume de Uso das Estações
start_station_usage = trips.groupby('start_station_name').size().reset_index(name='start_count')
end_station_usage = trips.groupby('end_station_name').size().reset_index(name='end_count')

# Juntar os contadores de uso por estação
station_usage = pd.merge(start_station_usage, end_station_usage, left_on='start_station_name', right_on='end_station_name', how='outer')

# Preencher valores NaN com 0 e somar os totais
station_usage.fillna(0, inplace=True)
station_usage['total_usage'] = station_usage['start_count'] + station_usage['end_count']

# Etapa 4: Adicionar 'total_usage' ao DataFrame 'stations'
stations = pd.merge(stations, station_usage[['start_station_name', 'total_usage']], left_on='name', right_on='start_station_name', how='left')
stations['total_usage'].fillna(0, inplace=True)  # Preencher valores NaN com 0
stations.drop(columns=['start_station_name'], inplace=True)  # Remover a coluna auxiliar

In [18]:
# Etapa 5: Identificar Estações Cobertas
stations['covered_by'] = [[] for _ in range(len(stations))]

COVERAGE_RADIUS = 10  # Exemplo: 1 km

for index, station in stations.iterrows():
    station_lat = station['lat']
    station_lon = station['lon']
    
    for index_compare, compare_station in stations.iterrows():
        compare_lat = compare_station['lat']
        compare_lon = compare_station['lon']
        
        # Evitar calcular a distância da estação para ela mesma
        if index != index_compare:
            distance = calculate_distance(station_lat, station_lon, compare_lat, compare_lon)
            
            # Adicionar à lista 'covered_by' se atender aos critérios
            if distance <= COVERAGE_RADIUS and station['total_usage'] > compare_station['total_usage']:
                stations.at[index, 'covered_by'].append(compare_station['name'])

# Verificar os resultados
for index, row in stations.iterrows():
    print(f"Estação: {row['name']} cobre: {row['covered_by']}")


Estação: Lafayette Park (Laguna St at Washington St) cobre: []
Estação: Balboa Park (San Jose Ave at Sgt. John V. Young Ln cobre: []
Estação: Onondaga Ave at Alemany Blvd cobre: []
Estação: North Point St at Divisadero St cobre: []
Estação: Filbert St at Fillmore St cobre: []
Estação: Geneva Ave at Moscow St cobre: []
Estação: Allyne Park (Green St at Gough St) cobre: []
Estação: 13th St at Webster St cobre: []
Estação: Fillmore St at Jefferson St cobre: []
Estação: Glen Park BART Station cobre: []
Estação: 24th St at Bartlett St cobre: []
Estação: Otis St at Brady St cobre: []
Estação: Buchanan St at North Point St cobre: []
Estação: 2nd St at Folsom St cobre: ['Lafayette Park (Laguna St at Washington St)', 'Balboa Park (San Jose Ave at Sgt. John V. Young Ln', 'Onondaga Ave at Alemany Blvd', 'North Point St at Divisadero St', 'Filbert St at Fillmore St', 'Geneva Ave at Moscow St', 'Allyne Park (Green St at Gough St)', 'Fillmore St at Jefferson St', 'Glen Park BART Station', '24th St a

In [20]:
# Criar um DataFrame a partir das informações coletadas
covered_stations_data = []

for index, row in stations.iterrows():
    station_name = row['name']
    covered_by = row['covered_by']
    
    for covered_station in covered_by:
        covered_stations_data.append({'station_name': station_name, 'covered_by': covered_station})

covered_stations_df = pd.DataFrame(covered_stations_data)

# Exibir ou exportar o novo DataFrame
covered_stations_df


,station_name,covered_by
0,2nd St at Folsom St,Lafayette Park (Laguna St at Washington St)
1,2nd St at Folsom St,Balboa Park (San Jose Ave at Sgt. John V. Youn...
2,2nd St at Folsom St,Onondaga Ave at Alemany Blvd
3,2nd St at Folsom St,North Point St at Divisadero St
4,2nd St at Folsom St,Filbert St at Fillmore St
...,...,...
33437,Bancroft Way at College Ave,Virginia St at Shattuck Ave
33438,Bancroft Way at College Ave,Parker St at Fulton St
33439,Bancroft Way at College Ave,North Berkeley BART Station
33440,Bancroft Way at College Ave,Berkeley Civic Center


In [21]:
covered_stations_df.to_csv('covered_stations.csv', index=False)